<a href="https://colab.research.google.com/github/DhruvSharma036/NSFW-image-filter/blob/main/NSFW_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
print(os.listdir('/content/drive/MyDrive/NSFW image filter'))

print(os.listdir('/content/drive/MyDrive/NSFW image filter/resnet50_inappropriate_content_detect-master'))

print(os.listdir('/content/drive/MyDrive/NSFW image filter/resnet50_inappropriate_content_detect-master/test_images_adult'))
print(os.listdir('/content/drive/MyDrive/NSFW image filter/resnet50_inappropriate_content_detect-master/test_images_violence'))


['resnet50_inappropriate_content_detect-master']
['test_images_adult', 'test_images_violence', 'training_info_adult_model', 'resnet50_final_adult.h5', 'resnet50_train.py', 'test_model.py', 'training_info_cartoon_model', 'resnet50_final_cartoon.h5', 'README.md', 'training_info_violence_model', 'resnet50_final_violence.h5', 'dataset']
['normal', 'porn']
['normal', 'violence']


In [14]:
dataset_dir = '/content/drive/MyDrive/NSFW image filter/resnet50_inappropriate_content_detect-master'

# Defining paths for training
train_adult_dir = os.path.join(dataset_dir, 'test_images_adult')
train_violence_dir = os.path.join(dataset_dir, 'test_images_violence')

In [10]:
from keras.preprocessing.image import ImageDataGenerator

# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

# Training data generators
train_generator_adult = train_datagen.flow_from_directory(
    train_adult_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

train_generator_violence = train_datagen.flow_from_directory(
    train_violence_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 1999 images belonging to 2 classes.
Found 1056 images belonging to 2 classes.


In [3]:
!pip install tensorflow keras

In [12]:
from keras.applications import ResNet50
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam

# Load ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)

# Adding custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

# Compile the model
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Combine the generators
from itertools import chain

combined_generator = chain(train_generator_adult, train_generator_violence)

# Train the model
model.fit(
    combined_generator,
    steps_per_epoch=len(train_generator_adult) + len(train_generator_violence),
    epochs=10
)

# Save the model
model.save('/content/drive/MyDrive/resnet50_finetuned.h5')


94765736/94765736 [==============================] - 2s 0us/step
Epoch 1/10
96/96 [==============================] - 1006s 10s/step - loss: 0.6862 - accuracy: 0.6216
Epoch 2/10
96/96 [==============================] - 598s 6s/step - loss: 0.5959 - accuracy: 0.6656
Epoch 3/10
96/96 [==============================] - 595s 6s/step - loss: 0.5705 - accuracy: 0.6877
Epoch 4/10
96/96 [==============================] - 591s 6s/step - loss: 0.5877 - accuracy: 0.6735
Epoch 5/10
96/96 [==============================] - 596s 6s/step - loss: 0.5596 - accuracy: 0.6989
Epoch 6/10
96/96 [==============================] - 589s 6s/step - loss: 0.5636 - accuracy: 0.6975
Epoch 7/10
96/96 [==============================] - 595s 6s/step - loss: 0.5593 - accuracy: 0.6943
Epoch 8/10
96/96 [==============================] - 591s 6s/step - loss: 0.5560 - accuracy: 0.7024
Epoch 9/10
96/96 [==============================] - 594s 6s/step - loss: 0.5458 - accuracy: 0.7064
Epoch 10/10
96/96 [=======================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
from keras.models import load_model
import numpy as np
from keras.preprocessing.image import load_img, img_to_array

# Load the trained model
model = load_model('/content/drive/MyDrive/resnet50_finetuned.h5')

# Function to preprocess and predict
def preprocess_and_predict(image_path, model):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    return prediction

image_path = '/content/drive/MyDrive/NSFW image filter/resnet50_inappropriate_content_detect-master/test_images_adult/normal/4250575_7356090ba8.jpg'
prediction = preprocess_and_predict(image_path, model)
print(prediction)

1/1 [==============================] - 1s 1s/step
[[0.6587078  0.34129223]]
